# Summary

Welcome to the NASA Earthdata Cloud Clinic. This notebook will walk through two different access and transformation options available in the Earthdata Cloud: 

1. Direct s3 access using the [earthaccess](https://github.com/nsidc/earthaccess) python library
    * Working with [MEaSUREs Sea Surface Height Anomalies](https://podaac.jpl.nasa.gov/dataset/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205)
    * Discover data using [Earthdata Search](https://search.earthdata.nasa.gov/search/granules?p=C2270392799-POCLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=measures%20ssh%20anomalies&tl=1685392107!3!!)
2. Data subsetting using the [Harmony-py](https://github.com/nasa/harmony-py) python library
    * Requesting a subset of data from the [GHRSST Level 4 MUR Global Foundation Sea Surface Temperature Analysis (v4.1)](https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1) dataset using a vector-based geospatial file.
    * This dataset can also be viewed in [Earthdata Search](https://search.earthdata.nasa.gov/search?q=GHRSST%20Level%204%20MUR%20v4.1&ff=Available%20in%20Earthdata%20Cloud!Customizable). 
    
In both scenarios, we will be accessing data directly from Amazon Web Services (AWS), specifically in the us-west-2 region, which is where all cloud-hosted NASA Earthdata reside. This shared compute environment (JupyterHub) is also running in the same location. We will then load the data into Python as an `xarray` `dataset`.

See the bottom of the notebook for additional resources, including several tutorials that that served as a foundation for this clinic. 

## Learning Objectives

1. Utilize the `earthaccess` python library to search for data using spatial and temporal filters and explore search results
2. Perform in-region direct access of data from an Amazon Simple Storage Service (S3) bucket
3. Extract variables and spatial slices from an `xarray` dataset 
4. Plot data using `xarray` 
5. Conceptualize data subsetting services provided by NASA Earthdata, including Harmony
6. Plot a polygon geojson file with a basemap using `geoviews` 
7. Utilize the `harmony-py` library to request data over the Gulf of Mexico

---

# 1. Prerequisites

## AWS instance running in us-west-2

Data in NASA's Earthdata Cloud reside in Amazon Web Services (AWS) Simple Storage Service (S3) buckets. Access is provided via temporary credentials; this access is limited to requests made within the US West (Oregon) (code: us-west-2) AWS region. While this compute location is required for direct S3 access, all data in Earthdata Cloud are still freely available via download.  

## Earthdata Login

An Earthdata Login account is required to access data from the NASA Earthdata system. Please visit <https://urs.earthdata.nasa.gov> to register and manage your Earthdata Login account. This account is free to create and only takes a moment to set up.


---

## Import Required Packages

In [ ]:
# Direct access
import earthaccess 
from pprint import pprint
import xarray as xr

# Harmony
import geopandas as gpd
import geoviews as gv
gv.extension('bokeh', 'matplotlib')
from harmony import BBox, Client, Collection, Request, LinkType
import datetime as dt
import s3fs
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 1. Authentication for NASA Earthdata 

The first step is to get the correct authentication that will allow us to get cloud-hosted data from NASA. This is all done through Earthdata Login. The `login` method also gets the correct AWS credentials.

In [ ]:
auth = earthaccess.login(strategy="interactive", persist=True)

___
## 2. Accessing a NetCDF-4 file using S3 Direct Access

![](images/earthaccess-intro.png){fig-alt="Text says earthaccess python library is an open-source library to simplify Earthdata Cloud search and access" width=25%}

### Search for data

`earthaccess` leverages the [Common Metadata Repository (CMR) API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html) to search for collections and granules. Earthdata Search also uses the CMR API. Let's head back to our [Earthdata Search](https://search.earthdata.nasa.gov/search/granules?p=C2270392799-POCLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=measures%20ssh%20anomalies&tl=1685392107!3!!) results to gather more information about our dataset of interest. The dataset "short name" can be found by clicking on the Info button on our collection search result. 

In [ ]:
ssh_short_name = "SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205"

results = earthaccess.search_data(
    short_name=ssh_short_name,
    cloud_hosted=True,
    temporal=("2021-07-01", "2021-09-30"),
)

According to PO.DAAC's dataset [landing page](https://podaac.jpl.nasa.gov/dataset/SEA_SURFACE_HEIGHT_ALT_GRIDS_L4_2SATS_5DAY_6THDEG_V_JPL2205), gridded Sea Surface Height Anomalies (SSHA) above a mean sea surface are provided. The data are produced on a 1/6th degree grid every 5 days.

We can discover more information about the matching files:

In [ ]:
pprint(results[0])

### Direct In-region Access

Our code will work the same way if we are running it "in-region", within our shared cloud environment, or locally from our laptop.

Since we are working in the AWS `us-west-2` region, we can stream data directly to xarray. We are using the `open_mfdataset()` (multi-file) method, which is required when using `earthaccess`.

In [ ]:
ds = xr.open_mfdataset(earthaccess.open(results))
ds

We can create a subsetted xarray dataset by extracting the SLA variable and slicing the dataset by a smaller area of interest near the state of Baja, Mexico. 

In [ ]:
ds_subset = ds['SLA'].sel(Latitude=slice(15.8, 35.9), Longitude=slice(234.5,260.5)) 
ds_subset

Use the built-in plotting functino of xarray to create a plot of SLA standard deviation over time: 

In [ ]:
ds_subset.std('Time').plot(figsize=(10,6), x='Longitude', y='Latitude');

---

## 3. Accessing a subsetted NetCDF-4 file using Transformation Services in the Cloud

In addition to directly accessing the files archived and distributed by each of the NASA DAACs, many datasets also support services that allow us to customize the data via subsetting, reformatting, reprojection/regridding, and file aggregation. Using [Earthdata Search](https://search.earthdata.nasa.gov/search?q=GHRSST%20Level%204%20MUR%20v4.1&ff=Available%20in%20Earthdata%20Cloud!Customizable), we can find datasets that support these services using the "Customizable" filter. 

![](images/subsetting_diagram.png){fig-alt="Three maps of the United States are present, with a red bounding box over the state of Colorado. Filtering and subsetting are demonstrated by overlaying SMAP L2 data, with data overlapping and cropping the rectangle, respectively."  width=25%}

We will find, request, and open customized data using [Harmony](https://harmony.earthdata.nasa.gov/), below.

### Define area of interest

First, use `geopandas` to read in a geojson file containing a polygon feature over the Gulf of Mexico. The geojson file is found in the ~/data directory. 

In [ ]:
geojson_path = './data/gulf.json'
gdf = gpd.read_file(geojson_path) #Return a GeoDataFrame object

We can plot the polygon using the `geoviews` package that we imported as gv with ‘bokeh’ and ‘matplotlib’ extensions. The following has reasonable width, height, color, and line widths to view our polygon when it is overlayed on a base tile map.

In [ ]:
base = gv.tile_sources.EsriImagery.opts(width=650, height=500)
ocean_map = gv.Polygons(gdf).opts(line_color='yellow', line_width=5, color=None)
base * ocean_map

### Use Harmony-Py to request a spatial subset of data

First, we need to create a Harmony Client, which is what we will interact with to submit and inspect a data request to Harmony, as well as to retrieve results.

When creating the Client, we need to provide Earthdata Login credentials. This basic line below assumes that we have a `.netrc` available. See the Earthdata Cloud Cookbook [appendix]("https://nasa-openscapes.github.io/earthdata-cloud-cookbook/appendix/authentication.html") for more information on [Earthdata Login]("https://urs.earthdata.nasa.gov/") and netrc setup. 

In [ ]:
harmony_client = Client()

### Create Harmony Request

See the [harmony-py documentation](https://harmony-py.readthedocs.io/en/latest/) for details on how to construct your request.

In [ ]:
sst_short_name="MUR-JPL-L4-GLOB-v4.1"

request = Request(
    collection=Collection(id=sst_short_name),
    shape=geojson_path,
    temporal={
    'start': dt.datetime(2021, 8, 1, 1),
    'stop': dt.datetime(2021, 8, 1, 2)   
    },
)

### Submit request

Now that the request is created, we can now submit it to Harmony using the Harmony Client object. A job id is returned, which is a unique identifier that represents the submitted request.

In [ ]:
job_id = harmony_client.submit(request)
job_id

### Check request status

Depending on the size of the request, it may be helpful to wait until the request has completed processing before the remainder of the code is executed. The wait_for_processing() method will block subsequent lines of code while optionally showing a progress bar.

In [ ]:
harmony_client.wait_for_processing(job_id, show_progress=True)

### View Harmony job response and output URLs
Once the data request has finished processing, we can view details on the job that was submitted to Harmony, including the API call to Harmony, and informational messages on the request if available.

result_json() calls wait_for_processing() and returns the complete job in JSON format once processing is complete. 

In [ ]:
data = harmony_client.result_json(job_id)
pprint(data)

### Direct In-region Access

Just like above, the subsetted outputs produced by Harmony can be accessed directly from the cloud. 

#### Retrieve list of output URLs.

The result_urls() method calls wait_for_processing() and returns a list of the processed data URLs once processing is complete. You may optionally show the progress bar as shown below.

In [ ]:
results = harmony_client.result_urls(job_id, link_type=LinkType.s3)
urls = list(results)
url = urls[0]
print(url)

Using `aws_credentials` you can retrieve the credentials needed to access the Harmony s3 staging bucket and its contents.

In [ ]:
creds = harmony_client.aws_credentials()

#### Open staged files with *s3fs* and *xarray*

We use the AWS `s3fs` package to create a file system that can then be read by xarray:

In [ ]:
s3_fs = s3fs.S3FileSystem(
    key=creds['aws_access_key_id'],
    secret=creds['aws_secret_access_key'],
    token=creds['aws_session_token'],
    client_kwargs={'region_name':'us-west-2'},
)

Now that we have our s3 file system set, including our declared credentials, we'll use that to open the url, and read in the file through xarray. This extra step is needed because xarray cannot open the S3 location directly. Instead, the S3 file object is passed to xarray, in order to then open the dataset. 

In [ ]:
f = s3_fs.open(url, mode='rb')
ds = xr.open_dataset(f)
ds

As before, we use the xarray built in plotting function to create a simple plot along the x and y dimensions of the dataset. We can see that the data are subsetted to our polygon:

In [ ]:
ds.analysed_sst.plot();

## Additional Resources

### Tutorials

This clinic was based off of several notebook tutorials including those presented during [past workshop events](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/tutorials/), along with other materials co-created by the NASA Openscapes mentors:
* [2021 Earthdata Cloud Hackathon](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/)
* [2021 AGU Workshop](https://nasa-openscapes.github.io/2021-Cloud-Workshop-AGU/)
* [Accessing and working with ICESat-2 data in the cloud](https://github.com/nsidc/NSIDC-Data-Tutorials/tree/main/notebooks/ICESat-2_Cloud_Access)
* [Analyzing Sea Level Rise Using Earth Data in the Cloud](https://github.com/betolink/earthaccess-gallery/blob/main/notebooks/Sea_Level_Rise/SSL.ipynb)

### Cloud services

The examples used in the clinic provide an abbreviated and simplified workflow to explore access and subsetting options available through the Earthdata Cloud. There are several other options that can be used to interact with data in the Earthdata Cloud including: 

* [OPeNDAP](https://opendap.earthdata.nasa.gov/) 
    * Hyrax provides direct access to subsetting of NASA data using Python or your favorite analysis tool
    * Tutorial highlighting OPeNDAP usage: https://nasa-openscapes.github.io/earthdata-cloud-cookbook/how-tos/working-locally/Earthdata_Cloud__Data_Access_OPeNDAP_Example.html
* [Zarr-EOSDIS-Store](https://github.com/nasa/zarr-eosdis-store)
    * The zarr-eosdis-store library allows NASA EOSDIS Collections to be accessed efficiently by the Zarr Python library, provided they have a sidecar DMR++ metadata file generated. 
    * Tutorial highlighting this library's usage: https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/09_Zarr_Access.html 

### Support

* [Earthdata Forum](https://forum.earthdata.nasa.gov/)
    * User Services and community support for all things NASA Earthdata, including Earthdata Cloud
* [Earthdata Webinar series](https://www.earthdata.nasa.gov/learn/webinars-and-tutorials)
    * Webinars from DAACs and other groups across EOSDIS including guidance on working with Earthdata Cloud
    * See the [Earthdata YouTube channel](https://www.youtube.com/@NASAEarthdata/featured) for more videos 